In [6]:
import Pkg; Pkg.add("CSV")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
 Installed WeakRefStrings ─ v0.6.1
 Installed ArrayLayouts ─── v0.1.5
 Installed FillArrays ───── v0.8.2
 Installed MacroTools ───── v0.5.3
 Installed LazyArrays ───── v0.14.10
 Installed FilePathsBase ── v0.7.0
 Installed CSV ──────────── v0.5.18
  Updating `~/.julia/environments/v1.3/Project.toml`
  [336ed68f] + CSV v0.5.18
  Updating `~/.julia/environments/v1.3/Manifest.toml`
  [4c555306] + ArrayLayouts v0.1.5
  [336ed68f] + CSV v0.5.18
  [48062228] + FilePathsBase v0.7.0
  [1a297f60] + FillArrays v0.8.2
  [5078a376] + LazyArrays v0.14.10
  [1914dd2f] + MacroTools v0.5.3
  [ea10d353] + WeakRefStrings v0.6.1


In [7]:
# LOAD PACKAGES
# -------------
using JuMP  # Need to say it whenever we use JuMP
using GLPK  # Loading the GLPK module for using its solver
using CSV

# MODEL CONSTRUCTION
# ------------------
capstone = Model(with_optimizer(GLPK.Optimizer, tm_lim=30*1000)) # We also set the time limit, in milliseconds

┌ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1273


A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

In [8]:
# INPUT DATA
# ----------

# Data on student preferences
Rank = CSV.read("wk12-Rank.csv",header=false)
size(Rank)

# Data on Pillars
Pillar = CSV.read("wk12-Pillar.csv",header=false)
size(Pillar)

# Data on Lower and Upper Bound (min and max number of students needed for each project from each discipline)
LowerBound = CSV.read("wk12-LowerBound.csv",header=false)
UpperBound = CSV.read("wk12-UpperBound.csv",header=false)
size(LowerBound)
size(UpperBound)

# Other data
Students = 1:170 # students
Projects = 1:61  # projects
PillarId = 1:4   # pillars

1:4

In [9]:
# VARIABLES
# ---------

# Binary variable taking value 1 if the s student is allocated to project p
@variable(capstone, x[Students,Projects], Bin)  

# Binary variable taking value 1 if project p is launched
@variable(capstone, y[Projects], Bin)          

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, 1:61
And data, a 61-element Array{VariableRef,1}:
 y[1] 
 y[2] 
 y[3] 
 y[4] 
 y[5] 
 y[6] 
 y[7] 
 y[8] 
 y[9] 
 y[10]
 y[11]
 y[12]
 y[13]
 ⋮    
 y[50]
 y[51]
 y[52]
 y[53]
 y[54]
 y[55]
 y[56]
 y[57]
 y[58]
 y[59]
 y[60]
 y[61]

In [10]:
# CONSTRAINTS
# -----------

# Every student is allocated to a single project
@constraint(capstone, allocatestudent[i=Students], sum(x[i,j] for j = Projects)==1)

# A student is allocated to a project only if the project is launched
@constraint(capstone, allocateifoffered[i=Students,j=Projects],x[i,j]<=y[j])

# Number of student of the different discplines allocated to a project is within the min. and upp. bound
@constraint(capstone, lower[j=Projects,k=PillarId],sum(x[i,j]*Pillar[i,k] for i = Students) >= LowerBound[k,j]*y[j])
@constraint(capstone, upper[j=Projects,k=PillarId],sum(x[i,j]*Pillar[i,k] for i = Students) <= UpperBound[k,j]*y[j])

2-dimensional DenseAxisArray{ConstraintRef{Model,C,Shape} where Shape<:AbstractShape where C,2,...} with index sets:
    Dimension 1, 1:61
    Dimension 2, 1:4
And data, a 61×4 Array{ConstraintRef{Model,C,Shape} where Shape<:AbstractShape where C,2}:
 upper[1,1] : x[1,1] + x[2,1] + x[3,1] + x[4,1] + x[5,1] + x[6,1] + x[7,1] + x[8,1] + x[9,1] + x[10,1] + x[11,1] + x[12,1] + x[13,1] + x[14,1] + x[15,1] + x[16,1] + x[17,1] + x[18,1] + x[19,1] + x[20,1] + x[21,1] + x[22,1] + x[23,1] + x[24,1] + x[25,1] + x[26,1] + x[27,1] + x[28,1] + x[29,1] + x[30,1] + x[31,1] + x[32,1] + x[33,1] + x[34,1] + x[35,1] + x[36,1] + x[37,1] + x[38,1] + x[39,1] + x[40,1] + x[41,1] + x[42,1] + x[43,1] + x[44,1] + x[45,1] + x[46,1] + x[47,1] + x[48,1] + x[49,1] + x[50,1] + x[51,1] + x[52,1] + x[53,1] + x[54,1] + x[55,1] + x[56,1] + x[57,1] + x[58,1] + x[59,1] - 3 y[1] ≤ 0.0                                                               …  upper[1,4] : x[150,1] + x[151,1] + x[152,1] + x[153,1] + x[154,1] + x[155,1]

In [11]:
# OBJECTIVE
# ---------

# Maximize the total utility
@objective(capstone, Max, sum(Rank[i,j]*x[i,j] for i = Students, j = Projects))

# PRINT THE MODEL IN A HUMAN-READABLE FORMAT
# ------------------------------------------
println("The optimization problem to be solved is:")
print(capstone) # Shows the model constructed in a human-readable form

The optimization problem to be solved is:
Max 3 x[1,1] + 4 x[1,8] + x[1,9] + 2 x[1,14] + 8 x[1,20] + 5 x[1,23] + 6 x[1,31] + 9 x[1,37] + 7 x[1,42] + 10 x[1,54] + 7 x[2,1] + 5 x[2,7] + 6 x[2,12] + 3 x[2,14] + 8 x[2,20] + 9 x[2,22] + 4 x[2,25] + 10 x[2,37] + x[2,54] + 2 x[2,58] + x[3,2] + 5 x[3,7] + 7 x[3,8] + 8 x[3,9] + 4 x[3,14] + 3 x[3,30] + 6 x[3,31] + 10 x[3,37] + 9 x[3,54] + 2 x[3,58] + 4 x[4,4] + 10 x[4,7] + 5 x[4,8] + 6 x[4,9] + 9 x[4,10] + 2 x[4,23] + 3 x[4,37] + 8 x[4,40] + 7 x[4,46] + x[4,54] + 8 x[5,7] + 4 x[5,8] + 9 x[5,9] + 7 x[5,14] + 6 x[5,25] + 3 x[5,30] + 2 x[5,31] + 10 x[5,37] + x[5,54] + 5 x[5,58] + 8 x[6,7] + 9 x[6,8] + 10 x[6,9] + 5 x[6,21] + 3 x[6,30] + 4 x[6,33] + 7 x[6,37] + 6 x[6,40] + 2 x[6,46] + x[6,57] + 4 x[7,1] + 10 x[7,7] + 2 x[7,8] + 7 x[7,9] + 3 x[7,10] + 5 x[7,14] + 9 x[7,25] + x[7,31] + 6 x[7,46] + 8 x[7,58] + 10 x[8,1] + 8 x[8,7] + 7 x[8,8] + 9 x[8,12] + 3 x[8,21] + 2 x[8,22] + 4 x[8,23] + x[8,25] + 6 x[8,30] + 5 x[8,31] + 5 x[9,1] + 6 x[9,8] + 10 x[9

 x[159,1] + x[159,2] + x[159,3] + x[159,4] + x[159,5] + x[159,6] + x[159,7] + x[159,8] + x[159,9] + x[159,10] + x[159,11] + x[159,12] + x[159,13] + x[159,14] + x[159,15] + x[159,16] + x[159,17] + x[159,18] + x[159,19] + x[159,20] + x[159,21] + x[159,22] + x[159,23] + x[159,24] + x[159,25] + x[159,26] + x[159,27] + x[159,28] + x[159,29] + x[159,30] + x[159,31] + x[159,32] + x[159,33] + x[159,34] + x[159,35] + x[159,36] + x[159,37] + x[159,38] + x[159,39] + x[159,40] + x[159,41] + x[159,42] + x[159,43] + x[159,44] + x[159,45] + x[159,46] + x[159,47] + x[159,48] + x[159,49] + x[159,50] + x[159,51] + x[159,52] + x[159,53] + x[159,54] + x[159,55] + x[159,56] + x[159,57] + x[159,58] + x[159,59] + x[159,60] + x[159,61] = 1.0
 x[160,1] + x[160,2] + x[160,3] + x[160,4] + x[160,5] + x[160,6] + x[160,7] + x[160,8] + x[160,9] + x[160,10] + x[160,11] + x[160,12] + x[160,13] + x[160,14] + x[160,15] + x[160,16] + x[160,17] + x[160,18] + x[160,19] + x[160,20] + x[160,21] + x[160,22] + x[160,23] + x[16

 x[60,30] + x[61,30] + x[62,30] + x[63,30] + x[64,30] + x[65,30] + x[66,30] + x[67,30] + x[68,30] + x[69,30] + x[70,30] + x[71,30] + x[72,30] + x[73,30] + x[74,30] + x[75,30] + x[76,30] + x[77,30] + x[78,30] + x[79,30] + x[80,30] + x[81,30] + x[82,30] + x[83,30] + x[84,30] + x[85,30] + x[86,30] + x[87,30] + x[88,30] + x[89,30] + x[90,30] + x[91,30] + x[92,30] + x[93,30] + x[94,30] + x[95,30] + x[96,30] + x[97,30] + x[98,30] + x[99,30] + x[100,30] + x[101,30] + x[102,30] + x[103,30] + x[104,30] + x[105,30] + x[106,30] + x[107,30] + x[108,30] + x[109,30] + x[110,30] + x[111,30] + x[112,30] + x[113,30] + x[114,30] + x[115,30] + x[116,30] + x[117,30] + x[118,30] + x[119,30] + x[120,30] + x[121,30] - 2 y[30] ≥ 0.0
 x[122,30] + x[123,30] + x[124,30] + x[125,30] + x[126,30] + x[127,30] + x[128,30] + x[129,30] + x[130,30] + x[131,30] + x[132,30] + x[133,30] + x[134,30] + x[135,30] + x[136,30] + x[137,30] + x[138,30] + x[139,30] + x[140,30] + x[141,30] + x[142,30] + x[143,30] + x[144,30] + x[14

 x[3,15] - y[15] ≤ 0.0
 x[3,16] - y[16] ≤ 0.0
 x[3,17] - y[17] ≤ 0.0
 x[3,18] - y[18] ≤ 0.0
 x[3,19] - y[19] ≤ 0.0
 x[3,20] - y[20] ≤ 0.0
 x[3,21] - y[21] ≤ 0.0
 x[3,22] - y[22] ≤ 0.0
 x[3,23] - y[23] ≤ 0.0
 x[3,24] - y[24] ≤ 0.0
 x[3,25] - y[25] ≤ 0.0
 x[3,26] - y[26] ≤ 0.0
 x[3,27] - y[27] ≤ 0.0
 x[3,28] - y[28] ≤ 0.0
 x[3,29] - y[29] ≤ 0.0
 x[3,30] - y[30] ≤ 0.0
 x[3,31] - y[31] ≤ 0.0
 x[3,32] - y[32] ≤ 0.0
 x[3,33] - y[33] ≤ 0.0
 x[3,34] - y[34] ≤ 0.0
 x[3,35] - y[35] ≤ 0.0
 x[3,36] - y[36] ≤ 0.0
 x[3,37] - y[37] ≤ 0.0
 x[3,38] - y[38] ≤ 0.0
 x[3,39] - y[39] ≤ 0.0
 x[3,40] - y[40] ≤ 0.0
 x[3,41] - y[41] ≤ 0.0
 x[3,42] - y[42] ≤ 0.0
 x[3,43] - y[43] ≤ 0.0
 x[3,44] - y[44] ≤ 0.0
 x[3,45] - y[45] ≤ 0.0
 x[3,46] - y[46] ≤ 0.0
 x[3,47] - y[47] ≤ 0.0
 x[3,48] - y[48] ≤ 0.0
 x[3,49] - y[49] ≤ 0.0
 x[3,50] - y[50] ≤ 0.0
 x[3,51] - y[51] ≤ 0.0
 x[3,52] - y[52] ≤ 0.0
 x[3,53] - y[53] ≤ 0.0
 x[3,54] - y[54] ≤ 0.0
 x[3,55] - y[55] ≤ 0.0
 x[3,56] - y[56] ≤ 0.0
 x[3,57] - y[57] ≤ 0.0
 x[3,58] - 

 x[45,19] - y[19] ≤ 0.0
 x[45,20] - y[20] ≤ 0.0
 x[45,21] - y[21] ≤ 0.0
 x[45,22] - y[22] ≤ 0.0
 x[45,23] - y[23] ≤ 0.0
 x[45,24] - y[24] ≤ 0.0
 x[45,25] - y[25] ≤ 0.0
 x[45,26] - y[26] ≤ 0.0
 x[45,27] - y[27] ≤ 0.0
 x[45,28] - y[28] ≤ 0.0
 x[45,29] - y[29] ≤ 0.0
 x[45,30] - y[30] ≤ 0.0
 x[45,31] - y[31] ≤ 0.0
 x[45,32] - y[32] ≤ 0.0
 x[45,33] - y[33] ≤ 0.0
 x[45,34] - y[34] ≤ 0.0
 x[45,35] - y[35] ≤ 0.0
 x[45,36] - y[36] ≤ 0.0
 x[45,37] - y[37] ≤ 0.0
 x[45,38] - y[38] ≤ 0.0
 x[45,39] - y[39] ≤ 0.0
 x[45,40] - y[40] ≤ 0.0
 x[45,41] - y[41] ≤ 0.0
 x[45,42] - y[42] ≤ 0.0
 x[45,43] - y[43] ≤ 0.0
 x[45,44] - y[44] ≤ 0.0
 x[45,45] - y[45] ≤ 0.0
 x[45,46] - y[46] ≤ 0.0
 x[45,47] - y[47] ≤ 0.0
 x[45,48] - y[48] ≤ 0.0
 x[45,49] - y[49] ≤ 0.0
 x[45,50] - y[50] ≤ 0.0
 x[45,51] - y[51] ≤ 0.0
 x[45,52] - y[52] ≤ 0.0
 x[45,53] - y[53] ≤ 0.0
 x[45,54] - y[54] ≤ 0.0
 x[45,55] - y[55] ≤ 0.0
 x[45,56] - y[56] ≤ 0.0
 x[45,57] - y[57] ≤ 0.0
 x[45,58] - y[58] ≤ 0.0
 x[45,59] - y[59] ≤ 0.0
 x[45,60] - y[60

 x[87,6] - y[6] ≤ 0.0
 x[87,7] - y[7] ≤ 0.0
 x[87,8] - y[8] ≤ 0.0
 x[87,9] - y[9] ≤ 0.0
 x[87,10] - y[10] ≤ 0.0
 x[87,11] - y[11] ≤ 0.0
 x[87,12] - y[12] ≤ 0.0
 x[87,13] - y[13] ≤ 0.0
 x[87,14] - y[14] ≤ 0.0
 x[87,15] - y[15] ≤ 0.0
 x[87,16] - y[16] ≤ 0.0
 x[87,17] - y[17] ≤ 0.0
 x[87,18] - y[18] ≤ 0.0
 x[87,19] - y[19] ≤ 0.0
 x[87,20] - y[20] ≤ 0.0
 x[87,21] - y[21] ≤ 0.0
 x[87,22] - y[22] ≤ 0.0
 x[87,23] - y[23] ≤ 0.0
 x[87,24] - y[24] ≤ 0.0
 x[87,25] - y[25] ≤ 0.0
 x[87,26] - y[26] ≤ 0.0
 x[87,27] - y[27] ≤ 0.0
 x[87,28] - y[28] ≤ 0.0
 x[87,29] - y[29] ≤ 0.0
 x[87,30] - y[30] ≤ 0.0
 x[87,31] - y[31] ≤ 0.0
 x[87,32] - y[32] ≤ 0.0
 x[87,33] - y[33] ≤ 0.0
 x[87,34] - y[34] ≤ 0.0
 x[87,35] - y[35] ≤ 0.0
 x[87,36] - y[36] ≤ 0.0
 x[87,37] - y[37] ≤ 0.0
 x[87,38] - y[38] ≤ 0.0
 x[87,39] - y[39] ≤ 0.0
 x[87,40] - y[40] ≤ 0.0
 x[87,41] - y[41] ≤ 0.0
 x[87,42] - y[42] ≤ 0.0
 x[87,43] - y[43] ≤ 0.0
 x[87,44] - y[44] ≤ 0.0
 x[87,45] - y[45] ≤ 0.0
 x[87,46] - y[46] ≤ 0.0
 x[87,47] - y[47] ≤ 0.0


Excessive output truncated after 524330 bytes.

In [12]:
# SOLVE IT
# --------

status = optimize!(capstone)

In [13]:
# DISPLAY THE RESULTS
# -------------------

println("Objective value: ", JuMP.objective_value(capstone)) 
println("x = ", JuMP.value.(x), " y = ", JuMP.value.(y))

Objective value: 1448.0
x = 2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 1:170
    Dimension 2, 1:61
And data, a 170×61 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0 

 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 

 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 

 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 